In [1]:
#| default_exp components


In [2]:
#| export
import fasthtml.common as fh
from fh_flowbite.core import *
from fasthtml.common import Div, P, Span, FT, HighlightJS
from enum import Enum, auto
from fasthtml.components import Uk_select,Uk_input_tag,Uk_icon
from functools import partial
from itertools import zip_longest
from typing import Union, Tuple, Optional, Sequence
from fastcore.all import *
import copy, re, httpx, os
from pathlib import Path
# from mistletoe.html_renderer import HTMLRenderer
# from mistletoe.span_token import Image
from pathlib import Path
# import mistletoe
# from lxml import html, etree
from fasthtml.components import Uk_input_range
import fasthtml.components as fh_comp

In [3]:
#| hide
from fasthtml.jupyter import *
from IPython.display import HTML, Markdown, Image

flowbite_hdrs = (
    Script(src="https://unpkg.com/@tailwindcss/browser@4"),
    Link(
        rel="stylesheet",
        href="https://cdn.jsdelivr.net/npm/flowbite@3.1.2/dist/flowbite.min.css",
    ),
    Link(rel="stylesheet", href="/output.css"),
    Script(
        "if (localStorage.getItem('color-theme') === 'dark' || (!('color-theme' in localStorage) && window.matchMedia('(prefers-color-scheme: dark)').matches)) {\r\n        document.documentElement.classList.add('dark');\r\n    } else {\r\n        document.documentElement.classList.remove('dark')\r\n    }"
    ),
    # Script(src="/tailwind.config.js"),
)

flowbite_ftrs = [
    Script(src="https://cdn.jsdelivr.net/npm/flowbite@3.1.2/dist/flowbite.min.js"),
]

if 'server' in globals(): server.stop()

app,rt = fh.fast_app(
    pico=False, 
    hdrs=(
        flowbite_hdrs,
        HighlightJS(langs=["python", "javascript", "html", "css"]),
    ),
    ftrs=flowbite_ftrs,
    htmlkw=dict(cls="bg-white dark:bg-gray-950 text-white font-sans antialiased"), 
    nb_hdrs=False,
    static_path=os.path.abspath('.'))

server = JupyUvi(app,port=8880)
Show = partial(HTMX,port=8880, app=app)

In [4]:
#| export
class TextT(VEnum):
    """
    Text Styles from Flowbite UI
    """
    
    # Text Size
    xs, sm, base, lg, xl = 'text-xs', 'text-sm', 'text-base', 'text-lg', 'text-xl'
    text_2xl, text_3xl, text_4xl = 'text-2xl', 'text-3xl', 'text-4xl'
    text_5xl, text_6xl, text_7xl = 'text-5xl', 'text-6xl', 'text-7xl'
    text_8xl, text_9xl = 'text-8xl', 'text-9xl'
    
    # Text Weight
    thin, extralight, light = 'font-thin', 'font-extralight', 'font-light'
    normal, medium, semibold = 'font-normal', 'font-medium', 'font-semibold'
    bold, extrabold, black = 'font-bold', 'font-extrabold', 'font-black'
    
    # Text Style
    italic = 'italic'
    
    # Text Color
    gray = 'text-gray-500 dark:text-gray-400'
    muted = 'text-gray-500 dark:text-gray-400'
    primary = 'text-blue-600 dark:text-blue-500'
    secondary = 'text-purple-600 dark:text-purple-500'
    success = 'text-green-500 dark:text-green-400'
    warning = 'text-yellow-500 dark:text-yellow-400'
    error = 'text-red-600 dark:text-red-500'
    info = 'text-blue-500 dark:text-blue-400'
    
    # Text Alignment
    left, right, center = 'text-left', 'text-right', 'text-center'
    justify, start, end = 'text-justify', 'text-start', 'text-end'
    
    # Vertical Alignment
    top, middle, bottom = 'align-top', 'align-middle', 'align-bottom'
    
    # Text Decoration
    underline, line_through = 'underline', 'line-through'
    uppercase, lowercase = 'uppercase', 'lowercase'
    capitalize, normal_case = 'capitalize', 'normal-case'
    
    # Line Height
    leading_none = 'leading-none'
    leading_tight = 'leading-tight'
    leading_snug = 'leading-snug'
    leading_normal = 'leading-normal'
    leading_relaxed = 'leading-relaxed'
    leading_loose = 'leading-loose'
    
    # Letter Spacing
    tracking_tighter = 'tracking-tighter'
    tracking_tight = 'tracking-tight'
    tracking_normal = 'tracking-normal'
    tracking_wide = 'tracking-wide'
    tracking_wider = 'tracking-wider'
    tracking_widest = 'tracking-widest'
    
    # Text Wrapping
    truncate = 'truncate'
    whitespace_normal = 'whitespace-normal'
    whitespace_nowrap = 'whitespace-nowrap'
    whitespace_pre = 'whitespace-pre'
    whitespace_pre_line = 'whitespace-pre-line'
    whitespace_pre_wrap = 'whitespace-pre-wrap'
    
    # Other
    highlight = 'bg-yellow-200 dark:bg-yellow-800 text-black'

class TextPresets(VEnum):
    """
    Common Typography Presets for Flowbite UI
    """
    muted_sm = TextT.muted+TextT.sm
    muted_lg = TextT.muted+TextT.lg
    
    bold_sm = TextT.bold+TextT.sm
    bold_lg = TextT.bold+TextT.lg
    
    primary_bold = stringify((TextT.primary, TextT.bold))
    success_bold = stringify((TextT.success, TextT.bold))
    warning_bold = stringify((TextT.warning, TextT.bold))
    error_bold = stringify((TextT.error, TextT.bold))
    
    md_weight_sm = stringify((TextT.sm, TextT.medium))
    md_weight_muted = stringify((TextT.medium, TextT.muted))

In [5]:
#| export
class ButtonT(VEnum):
    """Button utility types for Flowbite components"""
    def _generate_next_value_(name, start, count, last_values): 
        return str2cls('button', name)
    
    # Base styles
    default = "text-white bg-blue-700 hover:bg-blue-800 focus:ring-4 focus:ring-blue-300 font-medium rounded-lg text-sm px-5 py-2.5 dark:bg-blue-600 dark:hover:bg-blue-700 focus:outline-none dark:focus:ring-blue-800"
    alternative = "py-2.5 px-5 text-sm font-medium text-gray-900 focus:outline-none bg-white rounded-lg border border-gray-200 hover:bg-gray-100 hover:text-blue-700 focus:z-10 focus:ring-4 focus:ring-gray-200 dark:focus:ring-gray-700 dark:bg-gray-800 dark:text-gray-400 dark:border-gray-600 dark:hover:text-white dark:hover:bg-gray-700"
    dark = "text-white bg-gray-800 hover:bg-gray-900 focus:outline-none focus:ring-4 focus:ring-gray-300 font-medium rounded-lg text-sm px-5 py-2.5 dark:bg-gray-800 dark:hover:bg-gray-700 dark:focus:ring-gray-700 dark:border-gray-700"
    light = "text-gray-900 bg-white border border-gray-300 focus:outline-none hover:bg-gray-100 focus:ring-4 focus:ring-gray-200 font-medium rounded-lg text-sm px-5 py-2.5 dark:bg-gray-800 dark:text-white dark:border-gray-600 dark:hover:bg-gray-700 dark:hover:border-gray-600 dark:focus:ring-gray-700"
    green = "focus:outline-none text-white bg-green-700 hover:bg-green-800 focus:ring-4 focus:ring-green-300 font-medium rounded-lg text-sm px-5 py-2.5 dark:bg-green-600 dark:hover:bg-green-700 dark:focus:ring-green-800"
    red = "focus:outline-none text-white bg-red-700 hover:bg-red-800 focus:ring-4 focus:ring-red-300 font-medium rounded-lg text-sm px-5 py-2.5 dark:bg-red-600 dark:hover:bg-red-700 dark:focus:ring-red-900"
    yellow = "focus:outline-none text-white bg-yellow-400 hover:bg-yellow-500 focus:ring-4 focus:ring-yellow-300 font-medium rounded-lg text-sm px-5 py-2.5 dark:focus:ring-yellow-900"
    purple = "focus:outline-none text-white bg-purple-700 hover:bg-purple-800 focus:ring-4 focus:ring-purple-300 font-medium rounded-lg text-sm px-5 py-2.5 dark:bg-purple-600 dark:hover:bg-purple-700 dark:focus:ring-purple-900"

    # Sizes
    xs = auto()  # Will generate "button-xs"
    sm = auto()  # Will generate "button-sm"
    base = auto()  # Will generate "button-base"
    lg = auto()  # Will generate "button-lg"
    xl = auto()  # Will generate "button-xl"


In [6]:
#| export
class FlexT(VEnum):
    """Flexbox utility types"""
    def _generate_next_value_(name, start, count, last_values): 
        return str2cls('flex', name)
    
    # Display
    flex = "flex"
    inline = "inline-flex"
    
    # Direction
    row = auto()  # Will generate "flex-row"
    row_reverse = auto()  # Will generate "flex-row-reverse"
    col = auto()  # Will generate "flex-col"
    col_reverse = auto()  # Will generate "flex-col-reverse"
    
    # Justify content
    justify_start = "justify-start"
    justify_end = "justify-end"
    justify_center = "justify-center"
    justify_between = "justify-between"
    justify_around = "justify-around"
    justify_evenly = "justify-evenly"
    
    # Align items
    items_start = "items-start"
    items_end = "items-end"
    items_center = "items-center"
    items_baseline = "items-baseline"
    items_stretch = "items-stretch"
    
    # Wrap
    wrap = auto()  # Will generate "flex-wrap"
    wrap_reverse = auto()  # Will generate "flex-wrap-reverse"
    nowrap = auto()  # Will generate "flex-nowrap" 

In [7]:
#| hide
import nbdev; nbdev.nbdev_export()